Nome:

Victor Hugo de Souza Martins
____
Turma:

2TIAR
____
RM:

99591

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

# Carregar a imagem da assinatura
signature_image_path = "cnh.webp"
signature_image = cv2.imread(signature_image_path, cv2.IMREAD_GRAYSCALE)

# Aplicar limiarização (threshold) para criar uma imagem binária
_, binary_signature = cv2.threshold(signature_image, 150, 255, cv2.THRESH_BINARY_INV)

# Encontrar contornos
contours, _ = cv2.findContours(binary_signature, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Criar uma máscara em branco para desenhar a assinatura
signature_mask = np.zeros_like(signature_image)

# Desenhar o maior contorno (assumindo que é a assinatura)
if contours:
    largest_contour = max(contours, key=cv2.contourArea)
    cv2.drawContours(signature_mask, [largest_contour], -1, 255, thickness=cv2.FILLED)

# Salvar a imagem da assinatura extraída
extracted_signature_path = "cnh.webp1.png"
cv2.imwrite(extracted_signature_path, signature_mask)

# Mostrar a imagem original e a assinatura extraída
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.title('Original Signature Image')
plt.imshow(signature_image, cmap='gray')

plt.subplot(1, 2, 2)
plt.title('Extracted Signature')
plt.imshow(signature_mask, cmap='gray')

plt.show()

extracted_signature_path


Aqui usei os dados da imagem para serem gravados e extraidos para ter uma visualização boa dos dados.
___

usando a biblioteca OpenCV.

In [ ]:
import easyocr
import sqlite3
import cv2
from google.colab.patches import cv2_imshow

# Carregar a imagem
image_path = "cnh.webp"
image = cv2.imread(image_path)

# Verificar se a imagem foi carregada corretamente
if image is None:
    raise FileNotFoundError(f"Imagem não encontrada: {image_path}")

# Inicializar o leitor OCR do easyocr
reader = easyocr.Reader(['pt'])

# Realizar a leitura OCR na imagem
results = reader.readtext(image_path)

# Exibir resultados brutos da OCR para depuração
print("Resultados brutos da OCR:")
for bbox, text, prob in results:
    print(f"Texto: {text}, Probabilidade: {prob}")

# Função para extrair texto das regiões de interesse (ROIs)
def extract_text_from_rois(results):
    extracted_data = []
    for bbox, text, prob in results:
        text = text.strip()
        # Debug print para verificar o texto detectado
        print(f"Detectado: {text} com probabilidade {prob}")
        extracted_data.append(text)
    return extracted_data

# Extrair os dados das ROIs
extracted_data = extract_text_from_rois(results)

# Printar os dados extraídos
print("Dados extraídos:")
for text in extracted_data:
    print(text)

# Verificar se os dados extraídos estão corretos
if not extracted_data:
    print("Nenhum dado foi extraído. Verifique as ROIs e o texto detectado pela OCR.")
else:
    # Conectar ao banco de dados SQLite (ou criar se não existir)
    db_path = "document_data.db"
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # Criar a tabela para armazenar os dados do documento (se não existir)
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS documentos (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        campo TEXT NOT NULL,
        valor TEXT NOT NULL
    )
    ''')

    # Inserir os dados extraídos na tabela
    for i, text in enumerate(extracted_data, start=1):
        field = f"Campo_{i}"
        print(f"Inserindo {field}: {text} no banco de dados")
        cursor.execute('''
        INSERT INTO documentos (campo, valor) VALUES (?, ?)
        ''', (field, text))

    # Salvar as mudanças e fechar a conexão
    conn.commit()
    conn.close()

# Mostrar a imagem processada (opcional)
cv2_imshow(image)


Verificar se os dados foram gravados:

In [ ]:
import sqlite3

# Caminho para o banco de dados SQLite
db_path = "document_data.db"

# Conectar ao banco de dados
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Selecionar todos os registros da tabela documentos
cursor.execute('SELECT * FROM documentos')

# Buscar todos os resultados
rows = cursor.fetchall()

# Imprimir os dados extraídos
print("Dados armazenados no banco de dados:")
for row in rows:
    id, campo, valor = row
    print(f"ID: {id}, Campo: {campo}, Valor: {valor}")

# Fechar a conexão
conn.close()
